In [182]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Analisis inicial
Analizamos los tipos de datos que muestra el dataset.

In [183]:
df_train = pd.read_csv("salario.csv")

In [184]:
df_train.head()

,edad,trabajo,estudios,estado-civil,trabajo.1,posicion-familiar,etnia,sexo,ganancias_inversiones,perdidas_inversiones,horas-trabajo_semana,pais-origen,salario
0,39,State-gov,Bachelors,Never-married,Adm-clerical,No-en-familia,Blanco,Hombre,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Marido,Blanco,Hombre,0,0,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,No-en-familia,Blanco,Hombre,0,0,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Marido,Negro,Hombre,0,0,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Mujer,Negro,Mujer,0,0,40,Cuba,<=50K


In [185]:
df_train.info()
print('Testing Features shape: ', df_train.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27998 entries, 0 to 27997
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   edad                   27998 non-null  int64 
 1   trabajo                27998 non-null  object
 2   estudios               27998 non-null  object
 3   estado-civil           27998 non-null  object
 4   trabajo.1              27998 non-null  object
 5   posicion-familiar      27998 non-null  object
 6   etnia                  27998 non-null  object
 7   sexo                   27998 non-null  object
 8   ganancias_inversiones  27998 non-null  int64 
 9   perdidas_inversiones   27998 non-null  int64 
 10  horas-trabajo_semana   27998 non-null  int64 
 11  pais-origen            27998 non-null  object
 12  salario                27998 non-null  object
dtypes: int64(4), object(9)
memory usage: 2.8+ MB
Testing Features shape:  (27998, 13)


In [186]:
print(df_train.nunique())


edad                      72
trabajo                    9
estudios                  16
estado-civil               7
trabajo.1                 15
posicion-familiar          6
etnia                      5
sexo                       2
ganancias_inversiones    117
perdidas_inversiones      90
horas-trabajo_semana      94
pais-origen               42
salario                    2
dtype: int64


Haciendo un análisis principal del dataset, podemos obserbar que de las columnas 1 a la 7, junto a la 11 y 12 se tratan de variables categóricas.
En cuanto a las variables númericas luego deberemos tratar algun tipo de normalización o estandarización.

# Verificación de filas duplicadas
Vamos a comprobar si existen filas duplicadas en nuestro dataset, lo cual podría afectar al rendimiento y sesgar nuestro modelo.

In [187]:

duplicadas = df_train.duplicated().sum()
print(f"Número total de filas duplicadas: {duplicadas}")


porcentaje_duplicadas = (duplicadas / len(df_train)) * 100
print(f"Porcentaje de filas duplicadas: {porcentaje_duplicadas:.2f}%")


print("\nPrimeras filas duplicadas encontradas:")
df_train[df_train.duplicated()].head()

Número total de filas duplicadas: 2722
Porcentaje de filas duplicadas: 9.72%

Primeras filas duplicadas encontradas:


,edad,trabajo,estudios,estado-civil,trabajo.1,posicion-familiar,etnia,sexo,ganancias_inversiones,perdidas_inversiones,horas-trabajo_semana,pais-origen,salario
352,33,Private,Bachelors,Married-civ-spouse,Exec-managerial,Marido,Blanco,Hombre,0,0,40,United-States,>50K
391,27,Private,Bachelors,Never-married,Craft-repair,No-en-familia,Blanco,Hombre,0,0,50,United-States,<=50K
563,24,Private,HS-grad,Never-married,Handlers-cleaners,Soltero-a,Negro,Mujer,0,0,40,United-States,<=50K
570,24,Private,HS-grad,Never-married,Craft-repair,Hijo-unico,Blanco,Hombre,0,0,40,United-States,<=50K
673,33,Private,Bachelors,Married-civ-spouse,Exec-managerial,Marido,Blanco,Hombre,0,0,40,United-States,>50K


Nos encontramos con un 9.69% de filas duplicadas no es extremo, pero sí lo bastante alto como para que sea necesario revisar que hacer con ello:

- ¿Qué representan las filas?
El kaggle de la practica nos muestra que una fila esta compuesta por "12 columnas con información relevante para poder predecir el salario de una persona."

- Si cada fila representa una observación única los duplicados probablemente son errores y deberías eliminarlos.

- Si las filas representan eventos que pueden repetirse legítimamente, los duplicados podrían ser válidos, y eliminarlos podría hacerte perder información.

Analizando el significado del dataset es posible que distintas personas reales compartan exactamente las mismas características en las columnas del dataset (trabajo, estado civil, posción familiar etc.), especialmente si las variables son categóricas. Pero se son todas las columnas se repitan, tanto edad como inversiones o horas de trabajo ya es mas complicado que surga, así que vamos a verlo más en profundidad:

In [188]:
# Contamos cuántas veces se repite cada fila completa
duplicados_por_fila = df_train.value_counts()

print("Distribución de filas duplicadas:")
print("\nPrimeras 10 filas más repetidas:")
duplicados_por_fila.head(10)

Distribución de filas duplicadas:

Primeras 10 filas más repetidas:


edad  trabajo  estudios      estado-civil        trabajo.1          posicion-familiar  etnia   sexo    ganancias_inversiones  perdidas_inversiones  horas-trabajo_semana  pais-origen    salario
39    Private  HS-grad       Married-civ-spouse  Craft-repair       Marido             Blanco  Hombre  0                      0                     40                    United-States  <=50K      13
29    Private  HS-grad       Married-civ-spouse  Craft-repair       Marido             Blanco  Hombre  0                      0                     40                    United-States  <=50K      12
33    Private  HS-grad       Married-civ-spouse  Craft-repair       Marido             Blanco  Hombre  0                      0                     40                    United-States  <=50K      12
35    Private  HS-grad       Married-civ-spouse  Craft-repair       Marido             Blanco  Hombre  0                      0                     40                    United-States  <=50K      12
19    ?    

Como podemos ver la máxima repetición de una fila se da 13 veces, siendo una fila donde la edad, las inversiones son 0 en ambos casos y las horas de trabajo a la semana son las mismas.

Y esto se da en las 10 primeras filas más repetidas, por lo que sabiendo esto, una opción es optar a que no es necesario eliminar las filas. Ya que es razonable pensar dos o más personas que estudiaron lo mismo, trabajaben en lo mismo... es muy probable que ganen parecido siendo los dos ambos <=50K o >50k. Pero a su vez que compartan las variables númericas, asi como sexo, etnia, posición familiar, etc. es muy complicado. Tendiendo más a eliminación de dichas filas.

Como solución al problema vamos a ver eliminar los duplicados cambia la distribución del target:

In [189]:
print(df_train['salario'].value_counts(normalize=True))

df_sin_dup = df_train.drop_duplicates()

print(df_sin_dup['salario'].value_counts(normalize=True))

salario
<=50K    0.760411
>50K     0.239589
Name: proportion, dtype: float64
salario
<=50K    0.753838
>50K     0.246162
Name: proportion, dtype: float64


Las proporciones del target (“salario”) apenas cambian después de eliminar los duplicados, eso nos indica que los duplicados no aportan nueva información.

Dado que:
- Los duplicados son idénticos en todas las columnas.
- No representan un gran porcentaje del dataset.
- La distribución del target no cambia al eliminarlos.
- Y no encontramos una buena razón para mantener los resultados.

Decidimos que lo mejor es eliminarlo.

In [190]:

df_train = df_train.drop_duplicates()
print(f"Filas en el dataset déspues de eliminar: {len(df_train)}")

Filas en el dataset déspues de eliminar: 25276


# Limpieza de datos - Missing values
Buscamos y analizamos el porcentaje de NANs( valores vacios) que se encuentran en dataset, luego trataremos estos valores aplicando una estrategia:


Lo primero que nos fijamos es que la forma del dataset para representar los valores vacios, es a través del cáracter "?". Por lo que procederemos a verdaderamente vaciar dichos valores.

In [191]:
df_train.replace(r'^\s*\?\s*$', np.nan, regex=True, inplace=True)

In [192]:
print(df_train.isna().sum())

edad                        0
trabajo                  1408
estudios                    0
estado-civil                0
trabajo.1                1413
posicion-familiar           0
etnia                       0
sexo                        0
ganancias_inversiones       0
perdidas_inversiones        0
horas-trabajo_semana        0
pais-origen               493
salario                     0
dtype: int64


In [193]:
nan_por_columna = df_train.isnull().mean() * 100
pd.set_option('display.max_rows', None)
print(nan_por_columna[nan_por_columna > 0])

trabajo        5.570502
trabajo.1      5.590283
pais-origen    1.950467
dtype: float64


Solo 3 columnas (trabajo, trabajo.1 y pais-origen) poseen valores nulos.

In [194]:
nan_por_fila = df_train.isna().sum(axis=1)
print(nan_por_fila[nan_por_fila > 0].head(20))

14     1
27     2
38     1
51     1
61     3
69     2
77     2
93     1
106    2
128    2
149    2
154    2
160    2
187    2
201    2
221    2
226    2
243    2
245    1
249    1
dtype: int64


Hay filas que solo

In [195]:
porcentaje_filas_nan = (df_train.isna().any(axis=1).mean()) * 100
print(f"Porcentaje de filas con algún un NaN: {porcentaje_filas_nan:.2f}%")

Porcentaje de filas con algún un NaN: 7.45%


Estrategia para tratar los valores faltantes

Para tratar los valores faltantes, seguiremos la siguiente estrategia:

1. Para variables numéricas:
   - No es necesario tratar nada ya que los NANs solo aparecen en tres columnas categóricas
   
2. Para variables categóricas:
   - Debido a la proporción de filas con al menos un NANs, creemos que ese 7,3% es lo sufientemente notorio como para evitar eliminar todas esas filas, y buscar el imputar los valores, pero a su vez no es una gran parte del dataset por lo que podríamos prescindir de algunas filas.
   - Con esto en mente, hemos optado por usar dos estrategias al mismo tiempo:
        - Para las filas con 2 o más nulos. --> Se eliminan.
        - Para las filas con solo un nulo, rellenaremos ese valor a través de la moda (valor más frecuente) de su columna categórica.
   

Esta estrategia nos ayudará a mantener la distribución de los datos sin introducir sesgos significativos, a su vez que se eliminan aquellas filas con mucha información perdida.

In [196]:
# Primero, mantenemos solo las filas que tienen al menos (n_columnas - 1) valores no nulos
# Es decir: elimina las que tienen 2 o más NaN
df_temp = df_train.dropna(thresh=df_train.shape[1] - 1)

In [197]:
# Segundo, separamos las columnas numéricas y categóricas
numeric_features = df_temp.select_dtypes(include=['int64']).columns.tolist()
categorical_features = df_temp.select_dtypes(include=['object']).columns.tolist()

print("Características numéricas:", numeric_features)
print("\nCaracterísticas categóricas:", categorical_features)

Características numéricas: ['edad', 'ganancias_inversiones', 'perdidas_inversiones', 'horas-trabajo_semana']

Características categóricas: ['trabajo', 'estudios', 'estado-civil', 'trabajo.1', 'posicion-familiar', 'etnia', 'sexo', 'pais-origen', 'salario']


A la hora de rellenar los NANs en las columnas, a su vez podemos aprovechar para realizar la codificación one-hot que hace columna binaria por categoría, ya que la inmensa mayoria de los modelos "solo entienden números" y no acepta operar directamente con texto.

¿Porque codificación One-hot?

Debido a que si las transformasemos a números los dintintos valores de una columna categórica como por ejemplo "trabajo":
Private → 1
State-gov → 2
Self-emp → 3
Los modelo interpretaría que “State-gov” está entre “Private” y “Self-emp”, lo cual no tiene sentido no hay orden natural entre categorías, solo serviría para columnas donde esta relacion si exista, y posibles valores son por ejemplo “bajo”, “medio”, “alto”.

El One-hot al crear columna binaria por categoría, rompe esta posible relación que no aplica.

In [198]:
# pipeline de preprocesamiento solo para variables categóricas
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))  # Usamos OneHotEncoder para transformar los datos categóricos
])

# Aplicamos la transformación solo a las columnas categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'  # Esto mantiene las columnas numéricas sin modificar
)

# Ajustamos y transformamos los datos
transformed = preprocessor.fit_transform(df_temp)
print(transformed.shape)

(23868, 105)


In [199]:
encoder = preprocessor.named_transformers_['cat'].named_steps['encoder']
encoded_cols = encoder.get_feature_names_out(categorical_features)
all_columns = list(encoded_cols) + numeric_features

print(all_columns)  # Número total de columnas después de la transformación

['trabajo_ Federal-gov', 'trabajo_ Local-gov', 'trabajo_ Never-worked', 'trabajo_ Private', 'trabajo_ Self-emp-inc', 'trabajo_ Self-emp-not-inc', 'trabajo_ State-gov', 'trabajo_ Without-pay', 'estudios_ 10th', 'estudios_ 11th', 'estudios_ 12th', 'estudios_ 1st-4th', 'estudios_ 5th-6th', 'estudios_ 7th-8th', 'estudios_ 9th', 'estudios_ Assoc-acdm', 'estudios_ Assoc-voc', 'estudios_ Bachelors', 'estudios_ Doctorate', 'estudios_ HS-grad', 'estudios_ Masters', 'estudios_ Preschool', 'estudios_ Prof-school', 'estudios_ Some-college', 'estado-civil_ Divorced', 'estado-civil_ Married-AF-spouse', 'estado-civil_ Married-civ-spouse', 'estado-civil_ Married-spouse-absent', 'estado-civil_ Never-married', 'estado-civil_ Separated', 'estado-civil_ Widowed', 'trabajo.1_ Adm-clerical', 'trabajo.1_ Armed-Forces', 'trabajo.1_ Craft-repair', 'trabajo.1_ Exec-managerial', 'trabajo.1_ Farming-fishing', 'trabajo.1_ Handlers-cleaners', 'trabajo.1_ Machine-op-inspct', 'trabajo.1_ Other-service', 'trabajo.1_ P

In [200]:
df_transformed = pd.DataFrame(transformed, columns=all_columns)
df_transformed.head()

,trabajo_ Federal-gov,trabajo_ Local-gov,trabajo_ Never-worked,trabajo_ Private,trabajo_ Self-emp-inc,trabajo_ Self-emp-not-inc,trabajo_ State-gov,trabajo_ Without-pay,estudios_ 10th,estudios_ 11th,...,pais-origen_ Trinadad&Tobago,pais-origen_ United-States,pais-origen_ Vietnam,pais-origen_ Yugoslavia,salario_ <=50K,salario_ >50K,edad,ganancias_inversiones,perdidas_inversiones,horas-trabajo_semana
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,39.0,2174.0,0.0,40.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,50.0,0.0,0.0,13.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0,0.0,38.0,0.0,0.0,40.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,1.0,0.0,53.0,0.0,0.0,40.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,28.0,0.0,0.0,40.0


Hay que eliminar la columna salario_<=50k ya que es el inverso de salrio_>50K y ordenamos el target a la última columna

In [201]:
df_transformed=df_transformed.drop('salario_ <=50K', axis=1)
col = df_transformed.pop('salario_ >50K')  
df_transformed['salario_ >50K'] = col
df_transformed.head()

C:\Users\jaime\AppData\Local\Temp\ipykernel_13180\1490506385.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_transformed['salario_ >50K'] = col


,trabajo_ Federal-gov,trabajo_ Local-gov,trabajo_ Never-worked,trabajo_ Private,trabajo_ Self-emp-inc,trabajo_ Self-emp-not-inc,trabajo_ State-gov,trabajo_ Without-pay,estudios_ 10th,estudios_ 11th,...,pais-origen_ Thailand,pais-origen_ Trinadad&Tobago,pais-origen_ United-States,pais-origen_ Vietnam,pais-origen_ Yugoslavia,edad,ganancias_inversiones,perdidas_inversiones,horas-trabajo_semana,salario_ >50K
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,39.0,2174.0,0.0,40.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,50.0,0.0,0.0,13.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,38.0,0.0,0.0,40.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,53.0,0.0,0.0,40.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,28.0,0.0,0.0,40.0,0.0


In [202]:
nan_por_columna=(df_transformed.isna().sum().sum())
print("Número de NANs:",nan_por_columna)

Número de NANs: 0


# Estandarizado